## Basic StateGraph

In [1]:
from typing import TypedDict

In [2]:
class SimpleState(TypedDict):
    count: int

In [3]:
def increment(state: SimpleState) -> SimpleState:
    return {
        "count": state["count"] + 1
    }

In [4]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66


In [5]:
from langgraph.graph import END, StateGraph

In [6]:
def should_continue(state):
    if (state["count"] < 5):
        return "continue"
    else:
        return "stop"

In [7]:
graph = StateGraph(SimpleState)

In [8]:
graph.add_node("increment", increment)

In [9]:
graph.add_conditional_edges("increment", should_continue, {
    "continue": "increment",
    "stop": END
})

In [10]:
graph.set_entry_point("increment")

In [11]:
app = graph.compile()

In [12]:
state = {
    "count": 0
}

In [13]:
result = app.invoke(state)

In [14]:
print(result)

{'count': 5}


## Complex StateGraph

In [15]:
from typing import TypedDict, List
from langgraph.graph import END, StateGraph

In [16]:
class ComplexState(TypedDict):
    count: int
    sum: int
    history: List[int]

In [17]:
def increment(state: ComplexState) -> ComplexState:
    new_count = state["count"] + 1
    return {
        "count": new_count,
        "sum": state["sum"] + new_count,
        "history": state["history"] + [new_count]
    }

In [18]:
graph = StateGraph(ComplexState)

In [19]:
graph.add_node("increment", increment)

In [20]:
graph.set_entry_point("increment")

In [21]:
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END,
    }
)

In [22]:
app = graph.compile()

In [23]:
state = {
    "count" : 0,
    "sum": 0,
    "history": []
}

In [24]:
result = app.invoke(state)

In [25]:
print(result)

{'count': 5, 'sum': 15, 'history': [1, 2, 3, 4, 5]}


## Annotated StateGraph

In [26]:
from typing import Annotated
import operator

In [27]:
class AnnotatedState(TypedDict):
    count: int
    sum: Annotated[int, operator.add]
    history: Annotated[List[int], operator.concat]

In [28]:
def increment(state: AnnotatedState) -> AnnotatedState:
    new_count = state['count'] + 1
    return {
        "count" : new_count,
        "sum": new_count,
        "history": [new_count]
    }

In [29]:
graph = StateGraph(AnnotatedState)

In [30]:
graph.add_node("increment", increment)

In [31]:
graph.set_entry_point("increment")

In [32]:
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END,
    }
)

In [33]:
app = graph.compile()

In [34]:
state = {
    "count": 0,
    "sum": 0,
    "history": []
}

In [35]:
result = app.invoke(state)

In [36]:
print(result)

{'count': 5, 'sum': 15, 'history': [1, 2, 3, 4, 5]}
